<a href="https://colab.research.google.com/github/atreydesai/yys-misinfo-scraper/blob/main/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install selenium
!pip install yt-dlp

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 172.0/172.0 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 33.1 MB/s eta 0:00:00


In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

def get_snopes_fact_check_urls(search_term,page_num):
    chrome_options = Options()
    chrome_options.add_argument("--headless")
    chrome_options.add_argument("--disable-gpu")
    chrome_options.add_argument("--no-sandbox")

    driver = webdriver.Chrome(options=chrome_options)

    url = f"https://www.snopes.com/search/?q={search_term}#gsc.tab=0&gsc.q={search_term}&gsc.page={page_num}"

    try:
        driver.get(url)
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CLASS_NAME, "gs-title"))
        )

        links = driver.find_elements(By.CLASS_NAME, "gs-title")

        fact_check_urls = []
        for link in links:
            href = link.get_attribute("href")
            if href and "https://www.snopes.com/fact-check" in href:
                fact_check_urls.append(href)

        return fact_check_urls

    except Exception as e:
        print(f"An error occurred: {e}")
        return []
    finally:
        driver.quit()

if __name__ == '__main__':
    all_urls = []
    for page_num in range(1, 10):
        urls = get_snopes_fact_check_urls("video", page_num)
        if urls:
            switch_count=True
            print(f"Found the following fact-check URLs on page {page_num}:")
            for url in urls:
                if(switch_count):
                  print(url)
                  all_urls.append(url)
                  switch_count=False
                else:
                  switch_count=True
        else:
            print(f"No fact-check URLs found on page {page_num}.")

    print("\nAll collected URLs:")
    print(all_urls)

Found the following fact-check URLs on page 1:
https://www.snopes.com/fact-check/video-bugs-pringles-chip/
https://www.snopes.com/fact-check/dog-train-tracks-tiktok-video/
https://www.snopes.com/fact-check/1908-olympics-video/
https://www.snopes.com/fact-check/voter-fraud-in-2016-primary/
https://www.snopes.com/fact-check/escape-from-buckingham-palace/
https://www.snopes.com/fact-check/trump-retweet-cowboys/
https://www.snopes.com/fact-check/baby-giraffe-video/
https://www.snopes.com/fact-check/trump-project-2025-roberts/
https://www.snopes.com/fact-check/ants-termites-standoff/
https://www.snopes.com/fact-check/europa-io-jupiter-video/
https://www.snopes.com/fact-check/baby-peacock-video/
https://www.snopes.com/fact-check/us-cargo-ship-houthi-missile/
https://www.snopes.com/fact-check/video-of-a-globular-bolt-real/
https://www.snopes.com/fact-check/microscope-video-bugs-oreo-cookie/
Found the following fact-check URLs on page 2:
https://www.snopes.com/fact-check/video-crowd-celebratin

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
from bs4 import BeautifulSoup
import requests
import re
import yt_dlp
from google.colab import drive
import shutil
import os
import subprocess
import json


def get_headline(url):
    try:
        response = requests.get(url)
        response.raise_for_status()

        soup = BeautifulSoup(response.content, 'html.parser')
        title_container = soup.find('section', class_='title-container')
        if title_container:
            headline_tag = title_container.find('h1')
            headline = headline_tag.text.strip() if headline_tag else None
            return headline
        else:
            return None

    except requests.exceptions.RequestException as e:
        print(f"Error fetching URL: {e}")
        return None
    except Exception as e:
        print(f"An unexpected error occurred: {e}")
        return None

def get_subheadline(url):
    try:
        response = requests.get(url)
        response.raise_for_status()

        soup = BeautifulSoup(response.content, 'html.parser')
        title_container = soup.find('section', class_='title-container')
        if title_container:
            subheadline_tag = title_container.find('h2')
            subheadline = subheadline_tag.text.strip() if subheadline_tag else None
            return subheadline
        else:
            return None

    except requests.exceptions.RequestException as e:
        print(f"Error fetching URL: {e}")
        return None
    except Exception as e:
        print(f"An unexpected error occurred: {e}")
        return None

def _get_social_media_platform(link):
    if not link:
        return None

    social_media_domains = {
        "tiktok.com": "tiktok",
        "youtube.com": "youtube",
        "twitter.com": "twitter",
        "x.com": "twitter",
        "facebook.com": "facebook",
        "reddit.com": "reddit",
        "instagram.com": "instagram",
    }

    for domain, platform_name in social_media_domains.items():
        if domain in link:
            return platform_name
    return None

def get_rating(url):
    try:
        response = requests.get(url)
        response.raise_for_status()

        soup = BeautifulSoup(response.content, 'html.parser')
        rating_div = soup.find('div', class_='rating_title_wrap')
        if rating_div:
            for child in rating_div.children:
                if child.string and child.string.strip():
                    return child.string.strip()
        return None

    except (requests.exceptions.RequestException, Exception) as e:
        print(f"Error getting rating: {e}")
        return None

def get_links_from_article_rail(url):
    try:
        response = requests.get(url)
        response.raise_for_status()

        soup = BeautifulSoup(response.content, 'html.parser')
        youtube_container = soup.find('div', class_='embed_container youtube_container')
        if youtube_container:
            iframe = youtube_container.find('iframe', attrs={'data-src': True})
            if iframe and 'youtube.com/embed' in iframe['data-src']:
                return [iframe['data-src']]
            iframe = youtube_container.find('iframe', attrs={'src': True})
            if iframe and 'youtube.com/embed' in iframe['src']:
              return [iframe['src']]

        article_rail_wrapper = soup.find('div', class_='article_rail_wrapper')
        if article_rail_wrapper:
            links = []
            for a_tag in article_rail_wrapper.find_all('a', href=True):
                links.append(a_tag['href'])
            return links
        else:
            return []

    except requests.exceptions.RequestException as e:
        print(f"Error fetching URL: {e}")
        return []
    except Exception as e:
        print(f"An error occurred: {e}")
        return []

def get_first_social_link(url):
    links = get_links_from_article_rail(url)
    if not links:
        return None

    social_media_patterns = [
        r'https?://(?:www\.)?tiktok\.com',
        r'https?://(?:www\.)?youtube\.com',
        r'https?://(?:www\.)?(?:twitter\.com|x\.com)',
        r'https?://(?:www\.)?facebook\.com',
        r'https?://(?:www\.)?reddit\.com',
        r'https?://(?:www\.)?instagram\.com',
    ]

    for link in links:
        for pattern in social_media_patterns:
            if re.match(pattern, link):
                return link

    return "No link associated with article found."


def download_video(url, output_filename="downloaded_video.mp4", verbose=False):
    try:
        ydl_opts = {
            'format': 'best',
            'outtmpl': 'temp_download.%(ext)s',
            'quiet': not verbose,
            'no_warnings': True,
        }

        with yt_dlp.YoutubeDL(ydl_opts) as ydl:
            info_dict = ydl.extract_info(url, download=True)
            downloaded_filename = ydl.prepare_filename(info_dict)
            duration = info_dict.get('duration')


        _, ext = os.path.splitext(downloaded_filename)
        if ext.lower() == '.mp4':
            os.rename(downloaded_filename, output_filename)
            return True, "Successfully downloaded and renamed to MP4.", duration

        command = [
            'ffmpeg',
            '-i', downloaded_filename,
            '-c:v', 'libx264',
            '-c:a', 'aac',
            '-strict', 'experimental',
            '-y',
             output_filename
        ]

        if verbose:
            print(f"Running ffmpeg command: {' '.join(command)}")

        result = subprocess.run(command, capture_output=True, text=True)

        if result.returncode != 0:
            error_message = f"FFmpeg conversion failed:\nReturn code: {result.returncode}\nStdout: {result.stdout}\nStderr: {result.stderr}"
            if verbose:
                print(error_message)
            os.remove(downloaded_filename)
            return False, error_message, None
        else:
            if verbose:
              print("FFmpeg conversion successful.")
            os.remove(downloaded_filename)
            return True, "Successfully downloaded and converted to MP4.", duration

    except yt_dlp.utils.DownloadError as e:
        return False, f"yt-dlp download error: {e}", None
    except FileNotFoundError:
        return False, "ffmpeg not found.  Please make sure ffmpeg is installed and in your system's PATH.", None
    except Exception as e:
        return False, f"An unexpected error occurred: {e}", None

def download_progress_hook(d):
    if d['status'] == 'downloading':
        print(f"Downloading: {d['_percent_str']} {d['_speed_str']} ETA: {d['_eta_str']}", end='\r')
    elif d['status'] == 'finished':
        print(f"\nDownloaded: {d['filename']}")


def delete_video(filename):
    if not filename:
        print("No filename provided for deletion.")
        return

    try:
        os.remove(filename)
        print(f"File '{filename}' deleted successfully.")
    except FileNotFoundError:
        print(f"File '{filename}' not found.")
    except OSError as e:
        print(f"Error deleting file '{filename}': {e}")

if __name__ == '__main__':
    try:
        drive.mount('/content/drive')
        drive_mounted = True
    except Exception as e:
        print(f"Error mounting Google Drive: {e}")
        drive_mounted = False

    video_count = 1
    results = []

    print(f"There are {len(all_urls)} videos to be processed.")

    for URL in all_urls:
        headline = get_headline(URL)
        subheadline = get_subheadline(URL)
        social_platform = _get_social_media_platform(get_first_social_link(URL))
        rating = get_rating(URL)
        social_link = get_first_social_link(URL)
        destination_path = None
        download_success = False
        duration = None

        if social_link and social_link != "No link associated with article found.":
            download_success, download_message, duration = download_video(social_link)
            if download_success and drive_mounted:
                destination_path = f'/content/drive/MyDrive/.Research/CLIP/yooyeon-atrey/videos/video_{video_count}'
                try:
                    shutil.copy('/content/downloaded_video.mp4', destination_path)
                    print(f"Video copied to {destination_path}")
                except Exception as e:
                    print(f"Error copying video: {e}")
                    destination_path = Non
        else:
            download_message = "No social media link found, skipping download."

        if destination_path is not None and duration is not None and duration <= 600:
            video_data = {
                'original_url': URL,
                'headline': headline,
                'subheadline': subheadline,
                'social_platform': social_platform,
                'rating': rating,
                'social_link': social_link,
                'download_success': download_success,
                'download_message': download_message,
                'drive_path': destination_path,
                'duration': duration,
            }
            results.append(video_data)
            video_count += 1
            if download_success:
                delete_video('/content/downloaded_video.mp4')
        else:
            #Do not add the video data to the results.
            print(f"Skipping video {video_count} due to invalid path, excessive duration, or download failure.")
            if download_success:
               delete_video('/content/downloaded_video.mp4')
            video_count += 1 # this will still increment to avoid conflicts

    with open("video_data.json", "w") as f:
        json.dump(results, f, indent=4)
    print("Video data saved to video_data.json")

    if drive_mounted:
        try:
            shutil.copy("video_data.json", "/content/drive/MyDrive/.Research/CLIP/yooyeon-atrey/videos/")
            print("JSON file copied to Google Drive.")
        except Exception as e:
            print(f"Error copying JSON file to Drive: {e}")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
There are 97 videos to be processed.
Video copied to /content/drive/MyDrive/.Research/CLIP/yooyeon-atrey/videos/video_1
File '/content/downloaded_video.mp4' deleted successfully.


ERROR: [TikTok] 7012463416216489218: Video not available, status code 10222; please report this issue on  https://github.com/yt-dlp/yt-dlp/issues?q= , filling out the appropriate issue template. Confirm you are on the latest version using  yt-dlp -U


Video copied to /content/drive/MyDrive/.Research/CLIP/yooyeon-atrey/videos/video_3
File '/content/downloaded_video.mp4' deleted successfully.
Video copied to /content/drive/MyDrive/.Research/CLIP/yooyeon-atrey/videos/video_4
File '/content/downloaded_video.mp4' deleted successfully.
Video copied to /content/drive/MyDrive/.Research/CLIP/yooyeon-atrey/videos/video_5
File '/content/downloaded_video.mp4' deleted successfully.


ERROR: [twitter] 1265855459719892993: No video could be found in this tweet


Video copied to /content/drive/MyDrive/.Research/CLIP/yooyeon-atrey/videos/video_7
File '/content/downloaded_video.mp4' deleted successfully.


...

ERROR: [twitter] 1785891276228895152: No video could be found in this tweet
ERROR: [twitter] 1867317812450246759: No video could be found in this tweet
ERROR: [facebook] pfbid0pjz18GG6YwH1mYUcvcviHFosdbjp3LBzmXwd5qzVzUTuvyLhJhjUj2JxrMxCk4dyl: Unable to download webpage: HTTP Error 404: Not Found (caused by <HTTPError 404: Not Found>)


ERROR: [youtube] BuO6yJrRAYw: Video unavailable


Video copied to /content/drive/MyDrive/.Research/CLIP/yooyeon-atrey/videos/video_79
File '/content/downloaded_video.mp4' deleted successfully.
Video copied to /content/drive/MyDrive/.Research/CLIP/yooyeon-atrey/videos/video_80
File '/content/downloaded_video.mp4' deleted successfully.
Video copied to /content/drive/MyDrive/.Research/CLIP/yooyeon-atrey/videos/video_81
File '/content/downloaded_video.mp4' deleted successfully.
Video copied to /content/drive/MyDrive/.Research/CLIP/yooyeon-atrey/videos/video_82
File '/content/downloaded_video.mp4' deleted successfully.
Video copied to /content/drive/MyDrive/.Research/CLIP/yooyeon-atrey/videos/video_83
File '/content/downloaded_video.mp4' deleted successfully.
Video copied to /content/drive/MyDrive/.Research/CLIP/yooyeon-atrey/videos/video_84
File '/content/downloaded_video.mp4' deleted successfully.
Video copied to /content/drive/MyDrive/.Research/CLIP/yooyeon-atrey/videos/video_86
File '/content/downloaded_video.mp4' deleted successfully.

ERROR: Unsupported URL: https://x.com/hashtag/Myanmar?src=hashtag_click
ERROR: [youtube] fQupQPCEw4U: Video unavailable
ERROR: [twitter] 1849549522185097530: No video could be found in this tweet


Video copied to /content/drive/MyDrive/.Research/CLIP/yooyeon-atrey/videos/video_91
File '/content/downloaded_video.mp4' deleted successfully.
Video copied to /content/drive/MyDrive/.Research/CLIP/yooyeon-atrey/videos/video_92
File '/content/downloaded_video.mp4' deleted successfully.
Video copied to /content/drive/MyDrive/.Research/CLIP/yooyeon-atrey/videos/video_93
File '/content/downloaded_video.mp4' deleted successfully.
Video copied to /content/drive/MyDrive/.Research/CLIP/yooyeon-atrey/videos/video_94
File '/content/downloaded_video.mp4' deleted successfully.
Video copied to /content/drive/MyDrive/.Research/CLIP/yooyeon-atrey/videos/video_95
File '/content/downloaded_video.mp4' deleted successfully.
Video copied to /content/drive/MyDrive/.Research/CLIP/yooyeon-atrey/videos/video_96
File '/content/downloaded_video.mp4' deleted successfully.
Video copied to /content/drive/MyDrive/.Research/CLIP/yooyeon-atrey/videos/video_97
File '/content/downloaded_video.mp4' deleted successfully.